In [37]:
"""
Preforming turbulent simulations

More text to follow....
"""

import numpy as np
import matplotlib.pyplot as plt
import aotools
import gen_turb_conditions
import ang_spec_multi_prop
%matplotlib qt

In [38]:
def sg_absorber(nx, ny, sig, n=16):
    
    rn = np.sqrt(nx**2 + ny**2)
    
    return np.exp(-(rn/sig)**n)

In [39]:
"""
Sinc-Gaussian model point source in polar co-ordinates
"""

def sinc_gauss(x, y, r, k, R, D):
    
    return np.exp( -1j * k/(2*R) * r**2) / D**2 * np.sinc(x/D) * np.sinc(y/D) * np.exp(-(r/(4*D))**2)

In [40]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return phi, rho

In [41]:
def circ(x, y, a): # can't get to work weirdly - now defunct -> aotools.functions.pupil.circle()
    
    z = np.zeros_like(x)
    
    rows = len(x[0])
    columns = len(x[:0])
    
    r = np.sqrt(x**2 + y**2)
    
    for j in range(columns):
        
        for i in range(rows):
 
            if (r[i,j] < a):
        
                z[i,j] = 1
            
            if (r[i,j] == a):
                
                z[i,j] = 0.5
                
            if (r[i,j] > a):
                
                z[i,j] = 0
               
    return z

In [42]:
"""
Big list of parameters ...probably more to come
"""

l0 = 1e-20#inner scale [m]
L0 = 1e20 #approx 'inf' outer scale [m]

#L0 = 100.0 #Length of outer scale [m] - typically 100 m
#l0 = 0.01 #Length of inner scale [m] - typically 1-100 mm

D2 = 0.5 #diameter of the observation aperture [m]
wvl = 1e-6 #optical wavelength [m]
k = 2*np.pi / wvl #optical wavenumber [rad/m]
Dz = 50e3 #propagation distance [m]

#use sinc to model pt source
DROI = 4*D2
D1 = wvl*Dz / DROI

delta1 = 10e-3 #spacing at source screen [m]
deltan = 10e-3 #spacing at observation [m]
N = 512 #dfft power of 2  efficiency etc
n = 11 #number of partial propagations (n+1 screens needed altogether)

R = Dz #radius curvature of wavefront - set 'inf' if plane wave

# switch from total distance to individual distances - position of each screen through not source (?)
z = np.arange(1,n+1) * Dz/n
z = np.insert(z,0,0) #add source position here at origin - Position of screens and distance from plane 1
n = int(z.shape[0]) #now n is total number of screens and there are n-1 partial propagations 
delta_z = np.diff(z) # array of propagation distances from plane i to plane i+1, not total propagation distance

alpha = z / z[-1] 

delta = (1 - alpha) * delta1 + alpha * deltan

In [43]:
Cn2 = 1.0e-16
r0scrn, A, B, r0sw, r0pw, rytov = gen_turb_conditions.gen_turb_conditions_func(Cn2, k, Dz, n) #pw = plane wave, sw = spherical wave

print(r0scrn,r0sw,r0pw,rytov)
#print(B - A.dot(r0scrn))

[3.10647639e-03 8.17935220e+00 1.15193775e+01 1.20279764e+01
 1.01448698e+01 6.07980753e+00 8.58045351e-02 8.19408877e+00
 8.14501286e+00 7.77323654e+00 6.99691527e+00 5.38363493e+00] 0.12664241226501954 0.07030689768053816 0.4364896658907703


In [44]:
x1, y1 = np.meshgrid( delta1*np.arange(-N/2,N/2), delta1*np.arange(-N/2,N/2) )
theta, r1 = cart2pol(x1,y1) 

pt = sinc_gauss(x1, y1, r1, k, R, D1)

In [45]:
sig = 0.47*N*delta1
sg = sg_absorber(x1, y1, sig)

In [46]:
pixelised_diameter = D2 / deltan
pixelised_radius = pixelised_diameter /2
size = len(x1)
mask = aotools.functions.pupil.circle(pixelised_radius, size)

plt.figure()
plt.imshow(mask)
plt.colorbar()

In [47]:
MCF2 = np.zeros ((N,N))

In [48]:
ang_spec_multi_prop.set_up_geometry(pt, delta, z, n)

In [49]:
Uouts = []
nreals = 1

for idxreal in range(0, nreals):
 
    t = [np.exp(1j*aotools.turbulence.phasescreen.ft_sh_phase_screen(r0scrn[idxscr], N, delta[idxscr], L0, l0, FFT=None, seed=None)) for idxscr in range(0, n-1)]
    
    xn, yn, Uout = ang_spec_multi_prop.ang_spec_multi_prop_func(pt, wvl, delta1, deltan, z, sg, t)
    
    Uout = Uout * np.exp(-1j * np.pi /(wvl*R) * (xn**2 + yn**2)) #Collimate beam
    
    Uouts.append(Uout)
       
    MCF2 = MCF2 + ang_spec_multi_prop.corr2_ft(Uout, Uout, mask, deltan)#accumulate 2-D Mutual Coherence Function (un-normalised)
    
    
    #plt.figure()
    #plt.imshow( np.abs(Uout)**2 )
    #plt.colorbar()
    
    #plt.figure()
    #plt.imshow( np.angle(Uout) )
    #plt.colorbar()
    
MCDOC2 = np.abs(MCF2) / MCF2[int(N/2),int(N/2)]

#mask = circ(xn/D2, yn/D2, 1)

plt.figure()
plt.imshow(np.abs(MCDOC2))
plt.colorbar()

In [50]:
plt.figure()
plt.imshow( np.abs(Uout)**2 )
plt.colorbar()

In [51]:
plt.figure()
plt.imshow(np.angle(t[0]))
plt.colorbar()

In [52]:
del_r = np.arange(1,N) * deltan

vk = aotools.turbulence.slopecovariance.structure_function_vk(del_r, r0pw, L0)
kol = aotools.turbulence.slopecovariance.structure_function_kolmogorov(del_r, r0pw)

sf = [aotools.turbulence.calculate_structure_function(np.angle(phase_screen), nbOfPoint=N, step=None) for i, phase_screen in enumerate(t)]
sf = np.asarray(sf)
av_sf = sf.mean(axis = 0)

plt.figure()
plt.plot(abs(del_r)/r0pw ,vk, label = 'VK') 
plt.plot(abs(del_r)/r0pw ,kol, label = 'KOL') 
plt.legend()
plt.grid()

plt.figure()
for i,sfi in enumerate(sf):
    plt.plot(abs(del_r)/r0pw ,sfi ,'-.',label = ('Phase Screen ' + str(i) + ': r0 = ' + str( round(r0scrn[i],3) ) ) )
plt.legend()
plt.grid()

In [53]:
"""
Inspect middle row cross-section of intensity and phase
"""

rows,cols = MCDOC2.shape

mid_row = int(rows / 2)

MCDOC2_slice = MCDOC2[mid_row]

xn_slice = xn[mid_row]

plt.figure()
plt.plot(xn_slice/r0pw, np.abs(MCDOC2_slice), '.-')

In [54]:
#sh_sf = [aotools.turbulence.calculate_structure_function(sh_phase_screen, nbOfPoint=N, step=None) for i, sh_phase_screen in enumerate(sh_ps)]